In [167]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import mlflow
import sqlite3 as sql
from datetime import timedelta, datetime

In [202]:
conn = sql.connect('Application_prod.db')

data = pd.read_sql("""SELECT * 
                      FROM Raw_data""", conn)

data["date"] = pd.to_datetime(data["date"])
date = "20221001"
#today_date  = datetime.today()

today_date = datetime.strptime(date,"%Y%m%d")
last_date = today_date - timedelta(days=7)
focus_data = data.copy()[data['date'].between(last_date,today_date)]

print("---------------------------------------------------------------------------------")

df = focus_data.set_index('fullVisitorId')
df = df.copy()[df["country"].isin(["United States", "France", 
                                                "India", "China", "Germany",
                                                "Canada", "(not set)"])]
#change categorial columns
categorial = df[['channelGrouping','deviceCategory', 'country']]

for i in categorial.columns:
    df[i]= LabelEncoder().fit_transform(df[i])
    df[i].unique()
    
df = df.drop(['medium','date',"index"], axis= 1)
df = df.dropna(axis=0)

print("---------------------------------------------------------------------------------")

logged_model = 'runs:/1489eb854fb249f388e3404ab0b9fef4/model_xgboost'
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
# Predict on a Pandas DataFrame.
result = loaded_model.predict(pd.DataFrame(df))

print("---------------------------------------------------------------------------------")

data_result = focus_data.copy()[focus_data.country.isin(["United States", "France", 
                                                "India", "China", "Germany",
                                                "Canada", "(not set)"])].dropna()
data_result['Predict'] = result

result_to_save = data_result[["index","fullVisitorId","date","Predict"]]


print("---------------------------------------------------------------------------------")


try:
    
    check_date = pd.read_sql("SELECT * FROM Result_data",conn)
    print("Data existing wait to compare")
    
    check_date["date"] = pd.to_datetime(check_date["date"])
    if check_date[check_date["date"].between(last_date,today_date)].count()[0]!= 0 :
        print("Already Trained Data")
        if check_date[check_date["date"].between(last_date,today_date)].count()[0]<(result_to_save.count()[0]*0.85):
            pourcent_trained = round(check_date[check_date["date"].between(last_date,today_date)].count()[0] / result_to_save.count()[0] * 100,2)
            print(f"Data partialy to append check duplicate afterward.\n {pourcent_trained} % of the data already trained")
            result_to_save.to_sql("Result_data",conn, if_exists ="append",index=False)
        else: 
            print("Data trained to more than 85% data dropped")
            
    else:
        print("New Fresh Run append")
        result_to_save.to_sql("Result_data",conn, if_exists ="append",index=False)
        
        
except:
    print("first Run of infering creating table")
    
    result_to_save.to_sql("Result_data",conn, if_exists ="append",index=False)


---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
Data existing wait to compare
New Fresh Run append


In [190]:
df = focus_data.set_index('fullVisitorId')
df = df.copy()[df["country"].isin(["United States", "France", 
                                                "India", "China", "Germany",
                                                "Canada", "(not set)"])]
#change categorial columns
categorial = df[['channelGrouping','deviceCategory', 'country']]

for i in categorial.columns:
    df[i]= LabelEncoder().fit_transform(df[i])
    df[i].unique()
    

df = df.drop(['medium','date',"index"], axis= 1)
df = df.dropna(axis=0)

In [191]:
logged_model = 'runs:/1489eb854fb249f388e3404ab0b9fef4/model_xgboost'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
result = loaded_model.predict(pd.DataFrame(df))

In [192]:
data_result = focus_data.copy()[focus_data.country.isin(["United States", "France", 
                                                "India", "China", "Germany",
                                                "Canada", "(not set)"])].dropna()

In [193]:
data_result['Predict'] = result

In [194]:
result_to_save = data_result[["index","fullVisitorId","date","Predict"]]

In [195]:
try:
    
    check_date = pd.read_sql("SELECT * FROM Result_data",conn)
    print("Data existing wait to compare")
    
    check_date["date"] = pd.to_datetime(check_date["date"])
    if check_date[check_date["date"].between(last_date,today_date)].count()[0]!= 0 :
        print("Already Trained Data")
        if check_date[check_date["date"].between(last_date,today_date)].count()[0]<(result_to_save.count()[0]*0.85):
            pourcent_trained = round(check_date[check_date["date"].between(last_date,today_date)].count()[0] / result_to_save.count()[0] * 100,2)
            print(f"Data partialy to append check duplicate afterward.\n {pourcent_trained} % of the data already trained")
            result_to_save.to_sql("Result_data",conn, if_exists ="append",index=False)
        else: 
            print("Data trained to more than 85% data dropped")
            
    else:
        print("New Fresh Run append")
        result_to_save.to_sql("Result_data",conn, if_exists ="append",index=False)
        
        
except:
    print("first Run of infering creating table")
    
    result_to_save.to_sql("Result_data",conn, if_exists ="append",index=False)

Data existing wait to compare
Already Trained Data
Data partialy to append check duplicate afterward.
 42.8 % of the data already trained


In [139]:
check_date["date"] = pd.to_datetime(check_date["date"])
if check_date[check_date["date"].between(last_date,today_date)].count()[0]!= 0 :
    print("Already Trained Data")
    if check_date[check_date["date"].between(last_date,today_date)].count()[0]<(result_to_save.count()[0]*0.85):
        print("Data partialy append check duplicate")
        #result_to_save.to_sql("Result_data",conn, if_exists ="append",index=False)
    else: 
        print("Data trained to more than 80% data dropped")

else:
    print("new_clean_run")
    #result_to_save.to_sql("Result_data",conn, if_exists ="append",index=False)


Already Trained Data
Data trained to more than 80% data dropped


In [196]:
import sqlite3

#Connecting to sqlite
conn = sql.connect('Application_prod.db')

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Doping EMPLOYEE table if already exists
cursor.execute("DROP TABLE Result_data")
print("Table dropped... ")

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()

Table dropped... 


In [135]:
check_date[check_date["date"].between(last_date,today_date)].count()[0] !=0

True

In [129]:
check_date = pd.read_sql("SELECT * FROM Result_data",conn)

In [41]:
import datetime
year = str(20220919)[:4]
month = str(20220919)[4:6]
day = str(20220919)[6:]
date = f"{year}-{month}-{day}"
date_e = datetime.datetime.strptime(date, "%Y-%m-%d")

In [43]:
date_e

datetime.datetime(2022, 9, 19, 0, 0)

In [31]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47014 entries, 0 to 48053
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fullVisitorId    47014 non-null  uint64 
 1   bounces          47014 non-null  int64  
 2   time_on_site     47014 non-null  int64  
 3   pageviews        47014 non-null  float64
 4   medium           47014 non-null  object 
 5   channelGrouping  47014 non-null  object 
 6   deviceCategory   47014 non-null  object 
 7   country          47014 non-null  object 
 8   date             47014 non-null  int64  
 9   Predict          47014 non-null  int32  
dtypes: float64(1), int32(1), int64(3), object(4), uint64(1)
memory usage: 4.8+ MB
